# Internationalization (i18n) - Interactive Learning
# التدويل - تعلم تفاعلي

This notebook covers:
- Language detection strategies
- Translation management
- RTL (Right-to-Left) support
- Interpolation and formatting

يغطي هذا المفكرة:
- استراتيجيات كشف اللغة
- إدارة الترجمات
- دعم RTL (من اليمين لليسار)
- الاستيفال والتنسيق

## Part 1: Language Detection
## الجزء 1: كشف اللغة

In [ ]:
# Language detection implementation
from typing import Dict, Optional
from enum import Enum

class Language(str, Enum):
    ENGLISH = "en"
    ARABIC = "ar"
    SPANISH = "es"
    FRENCH = "fr"
    GERMAN = "de"
    CHINESE = "zh"
    JAPANESE = "ja"

def detect_language(
    headers: Dict[str, str],
    query_params: Dict[str, str],
    user_preference: Optional[str] = None,
) -> Language:
    """
    Detect language with priority-based strategy.
    
    Priority:
    1. Query parameter (?lang=ar)
    2. Accept-Language header
    3. User preference (saved in database)
    4. Content analysis (fallback)
    """
    # Priority 1: Query parameter
    if query_params and "lang" in query_params:
        try:
            return Language(query_params["lang"].lower())
        except ValueError:
            pass
    
    # Priority 2: Accept-Language header
    if headers and "Accept-Language" in headers:
        accept_lang = headers["Accept-Language"]
        for lang_part in accept_lang.split(","):
            lang_code = lang_part.split(";")[0].strip().lower()
            try:
                return Language(lang_code)
            except ValueError:
                pass
    
    # Priority 3: User preference
    if user_preference:
        try:
            return Language(user_preference)
        except ValueError:
            pass
    
    # Default to English
    return Language.ENGLISH

# Test cases
test_cases = [
    # Query parameter
    {"headers": {}, "query_params": {"lang": "ar"}, "expected": Language.ARABIC},
    {"headers": {}, "query_params": {"lang": "es"}, "expected": Language.SPANISH},
    # Header
    {"headers": {"Accept-Language": "ar-SA"}, "query_params": {}, "expected": Language.ARABIC},
    {"headers": {"Accept-Language": "en-US,es;q=0.9"}, "query_params": {}, "expected": Language.ENGLISH},  # q=0.9 means Spanish preferred
]

print("Language Detection Tests:")
for i, test in enumerate(test_cases):
    detected = detect_language(test["headers"], test["query_params"])
    status = "✓" if detected == test["expected"] else "✗"
    print(f"  Test {i+1}: {status} - Detected: {detected}, Expected: {test['expected']}")

### Exercise 1: Implement Content-Based Detection
### تمرين 1: تنفيذ الكشف المستند على المحتوى

In [ ]:
# TODO: Implement content-based language detection
# This can use libraries like langdetect or polyglot

from typing import List, Tuple

# Sample texts in different languages
samples = {
    Language.ENGLISH: "Hello, how are you today?",
    Language.ARABIC: "مرحبا، كيف حالك اليوم؟",
    Language.SPANISH: "Hola, ¿cómo estás hoy?",
    Language.FRENCH: "Bonjour, comment allez-vous aujourd'hui?",
}

def detect_from_content(text: str) -> Tuple[Language, float]:
    """
    Detect language from text content.
    Returns (language, confidence_score).
    
    TODO: Implement using langdetect or similar library
    """
    return (Language.ENGLISH, 0.8)  # Placeholder

# Test
print("\nContent-Based Detection (placeholder):")
for lang, sample in samples.items():
    detected, confidence = detect_from_content(sample)
    print(f"  {lang}: Detected {detected} (confidence: {confidence:.2%})")

### Solution / الحل

In [ ]:
# Solution using langdetect (if installed)
try:
    from langdetect import detect
    
    def detect_from_content(text: str) -> Tuple[Language, float]:
        """Detect language using langdetect library."""
        try:
            lang_code = detect(text)
            # Map to our Language enum
            lang_mapping = {
                "ar": Language.ARABIC,
                "en": Language.ENGLISH,
                "es": Language.SPANISH,
                "fr": Language.FRENCH,
                "de": Language.GERMAN,
            }
            language = lang_mapping.get(lang_code, Language.ENGLISH)
            return (language, 0.95)  # langdetect returns confidence
        except:
            return (Language.ENGLISH, 0.5)
    
    print("Langdetect-based detection available")
except ImportError:
    print("Langdetect not installed, using placeholder detection")

## Part 2: Translation Interpolation
## الجزء 2: الاستيفال في الترجمة

In [ ]:
# Translation interpolation examples

translations = {
    "welcome": "Welcome, {name}!",
    "document_count": "You have {count} documents",  # English
    "welcome_ar": "مرحبا، {name}!",
    "document_count_ar": "لديك {count} مستندات",  # Arabic
}

# Simple interpolation
print("Simple Interpolation:")
print(translations["welcome"].format(name="John"))
print(translations["document_count"].format(count=5))
print(translations["welcome_ar"].format(name="أحمد"))
print(translations["document_count_ar"].format(count=10))

In [ ]:
# Dictionary-based interpolation
def interpolate(template: str, **kwargs) -> str:
    """Interpolate template with keyword arguments."""
    result = template
    for key, value in kwargs.items():
        placeholder = "{" + key + "}"
        if placeholder in result:
            result = result.replace(placeholder, str(value))
    return result

templates = {
    "email_subject": "[{app}] {action} required: {item}",
    "email_body": "Your {item} {action} is pending. Please click the link below to proceed.",
}

print("\nDictionary Interpolation:")
email_msg = interpolate(
    templates["email_subject"],
    app="RAG Engine",
    action="verification",
    item="email address"
)
print(f"  Subject: {email_msg}")

## Part 3: RTL Support
## الجزء 3: دعم RTL

In [ ]:
# RTL (Right-to-Left) support utilities

RTL_LANGUAGES = ["ar", "he", "fa", "ur"]

def is_rtl(language_code: str) -> bool:
    """Check if language is RTL."""
    return language_code in RTL_LANGUAGES

def get_direction(language_code: str) -> str:
    """Get text direction."""
    return "rtl" if is_rtl(language_code) else "ltr"

def mirror_margin(value: int, direction: str) -> tuple:
    """Mirror margin value for RTL support."""
    if direction == "rtl":
        return ("auto", value)  # Flip for RTL
    return (value, "auto")  # Default for LTR

# Test
test_languages = ["en", "ar", "he", "fa", "es"]
print("RTL/LTR Detection:")
for lang in test_languages:
    is_rtl_val = is_rtl(lang)
    direction = get_direction(lang)
    margin = mirror_margin(20, direction)
    status = "RTL" if is_rtl_val else "LTR"
    print(f"  {lang}: {direction:4} ({status}), Margin: {margin}")

### Exercise 2: Flip Flexbox for RTL
### تمرين 2: عكس Flexbox لـ RTL

In [ ]:
# TODO: Implement Flexbox mirroring for RTL

def get_flexbox_align(language_code: str) -> str:
    """Get flex-direction and alignment for language."""
    if is_rtl(language_code):
        return "flex-direction: row-reverse; text-align: right;"
    return "flex-direction: row; text-align: left;"

# Generate CSS for different languages
for lang in ["en", "ar", "he"]:
    css = f"".{{lang}} {{ flexbox_align(lang) } }}""
    print(f"{lang}: {css}")

### Solution / الحل

In [ ]:
# Solution
def get_flexbox_align(language_code: str) -> str:
    """Get flex-direction and alignment for language."""
    if is_rtl(language_code):
        return "flex-direction: row-reverse; text-align: right; justify-content: flex-start;"
    return "flex-direction: row; text-align: left; justify-content: flex-start;"

# Generate CSS for different languages
print("Flexbox CSS (Solution):")
for lang in ["en", "ar", "he"]:
    css = f"".{lang} {{ flexbox_align(lang) } }}""
    print(f"  {lang}: {css}")

## Part 4: i18n Service Integration
## الجزء 4: تكامل خدمة التدويل

In [ ]:
# Mock i18n service for testing

class MockI18nService:
    def __init__(self):
        self._translations = {
            "en": {
                "welcome": "Welcome",
                "document_uploaded": "Document uploaded successfully",
                "document_failed": "Document upload failed",
                "search_no_results": "No results found",
            },
            "ar": {
                "welcome": "مرحبًا",
                "document_uploaded": "تم تحميل المستند بنجاح",
                "document_failed": "فشل تحميل المستند",
                "search_no_results": "لم يتم العثور على نتائج",
            },
        }
    
    def get_translation(self, language_code: str, key: str, **kwargs):
        lang_translations = self._translations.get(language_code, self._translations["en"])
        template = lang_translations.get(key, key)
        if kwargs:
            return template.format(**kwargs)
        return template

    def is_rtl(self, language_code: str) -> bool:
        return language_code in ["ar", "he", "fa", "ur"]

# Test
i18n = MockI18nService()

print("i18n Service Test:")
print(f"  English: {i18n.get_translation('en', 'welcome')}")
print(f"  Arabic: {i18n.get_translation('ar', 'welcome')}")
print(f"  Arabic RTL: {i18n.is_rtl('ar')}")
print(f"  English RTL: {i18n.is_rtl('en')}")

# Test interpolation
print(f"\n  English with interpolation: {i18n.get_translation('en', 'document_count', count=5)}")
print(f"  Arabic with interpolation: {i18n.get_translation('ar', 'document_count', count=10)}")

## Summary / الملخص

**Key concepts covered / المفاهيم الرئيسية المشمولة:**

1. **Language detection**: Priority-based strategy (query > header > user preference)
2. **Translation management**: Categorized keys with interpolation support
3. **RTL support**: Direction detection and layout mirroring
4. **i18n service**: Comprehensive translation management
5. **Content analysis**: Using libraries like langdetect for detection

**النقاط الرئيسية المشمولة:**

1. **كشف اللغة**: استراتيجية قائمة على الأولوية
2. **إدارة الترجمات**: مفاتح مصنفة مع دعم الاستيفال
3. **دعم RTL**: كشف الاتجاه وعكس التخطيط
4. **خدمة التدويل**: إدارة شاملة للترجمات
5. **تحليل المحتوى**: استخدام مكتبات مثل langdetect للكشف